# Report 2

## Recursive data structures

#### Marcin Kapiszewski 156048
#### Adam Tomys 156057

Group 2

### Compared recursive structures:

1. Sorted (singly) linked list:
    - It consists of nodes, each of them contains key, value, and a reference to the next node. 
    - Nodes are sorted by keys.
    - First element is called the head and the last element is called the tail.
    - Adding element requires going over the list to find the correct place, copying reference from the previous node and overwriting that reference with a reference to the new node. If the new node would be the first element it becomes a new head and copies the reference to the previous head. If it would become a new tail, the previous one would just save the new one's reference.
    - Deleting element just overwrites the reference contained by the previous node with the reference that is saved in a node that we are deleting. Unless it is a head then the next one just becomes a head, and if it is a tail we delete the reference in the previous node altogether.
2. Binary search tree:
    - It consists of nodes, each of them contains key, value, and references to the left and right child.
    - The left child's key is always lesser and the right child's is always greater than the parent's.
    - First element, at the top, is called the root of the tree.
    - Adding element requires us to start at the root. We compare the new node's key with the root's key. If it is lesser we go to the left child, otherwise to the right. we repeat that process until we go across a Null child (that does not exist), then the new Node becomes that child.
    - Deleting element: If the node to be deleted does not have children just delete the node. If it has one child just replace that node with the child. If it has 2 children replace its key and value with ones of the node with the least key in the right subtree, and remove that node from the tree.
3. AVL tree (optionally):
    - It is an extension of binary search tree. It requires each node's left and right subtree to have a diffrence between -1 and 1 in height. After each Addition or deletion it checks if a tree is unbalanced and then it uses one of the following 4 rotaition to rebalance the tree.
    - Left rotation: The right child becomes the new root and the original root becomes the left child of the new root.
    - Right rotation: The Left child becomes the new root and the original root becomes the right child of the new root.
    - Left-right rotation: Use left rotation on the left subtree than right rotation on the tree.
    - Right-left rotation: Use right rotation on the right subtree than left rotation on the tree.
    


### Implementation Difficulties:

1. Sorted (singly) linked list:
    - the simplest structure, we didn't had any problems with implementation.
    - firstly, we tried recursive implementation, but with big instance size our program was out of memory.
2. Binary search tree:
    - our implementation is recursive, so when we create a degenerate tree(by feeding the structure with increasing or decreasing instance type), we have a problem with crashing kernel, but for small instances it works fine(comparing instance size in our experiments is 5000). For our assignment bigger instance sizes weren't necessery, because we wanted to find trends and compare structures with different insert and remove instance type.
3. AVL tree (optionally):
    - AVL was the most complex structure to implement, a lot of code we copied from our BST(works fine with big instances of all type, because it doesn't allow tree to degenerate, so tree height isn't high)
    - it's just BST with additional node attribute and additional balancing logic, balancing logic was the most challenging part

In [16]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from Binary_search_tree import BinarySearchTree
from Single_linked_list import SingleLinkedList
from AVL_Tree import AVLTree
from Evaluate_program import measureTime
from Generate_input import generateIncreasing, generateRandom, generateDecreasing, generateVShape

In [18]:
structers = [BinarySearchTree, SingleLinkedList, AVLTree]

In [19]:
def functionToEvaluate(structClass, inputData, dataGenerator):
    struct = structClass()
    for elem in inputData:
        struct.insert(elem, elem)
    inputDataRemove = dataGenerator(data = inputData)
    for elem in inputDataRemove:
        struct.remove(elem)

In [20]:
inputTypes = [generateIncreasing, generateDecreasing, generateRandom, generateVShape]
def getTimes(structClass):
    print(structClass.__name__)
    insert_time_type = {}
    for insertInputType in inputTypes:
        print(f"\t{insertInputType.__name__}")
        inputData = insertInputType(5000)
        remove_time_type = {}
        for removeInputType in inputTypes:
            print(f"\t\t{removeInputType.__name__}")
            functionInput = [structClass, inputData, removeInputType]
            remove_time_type[removeInputType.__name__] = measureTime(functionToEvaluate, functionInput)
        insert_time_type[insertInputType.__name__] = remove_time_type
    return insert_time_type
        
        

In [21]:
times = {}
for struct in structers:
    times[struct.__name__] = getTimes(struct)

BinarySearchTree
	generateIncreasing
		generateIncreasing
		generateDecreasing
		generateRandom
		generateVShape
	generateDecreasing
		generateIncreasing
		generateDecreasing
		generateRandom
		generateVShape
	generateRandom
		generateIncreasing
		generateDecreasing
		generateRandom
		generateVShape
	generateVShape
		generateIncreasing
		generateDecreasing
		generateRandom
		generateVShape
SingleLinkedList
	generateIncreasing
		generateIncreasing
		generateDecreasing
		generateRandom
		generateVShape
	generateDecreasing
		generateIncreasing
		generateDecreasing
		generateRandom
		generateVShape
	generateRandom
		generateIncreasing
		generateDecreasing
		generateRandom
		generateVShape
	generateVShape
		generateIncreasing
		generateDecreasing
		generateRandom
		generateVShape
AVLTree
	generateIncreasing
		generateIncreasing
		generateDecreasing
		generateRandom
		generateVShape
	generateDecreasing
		generateIncreasing
		generateDecreasing
		generateRandom
		generateVShape
	generateRando

In [22]:
d_sum = {}
for struct, d in times.items():
    d_sum[struct] = sum([sum(d_2.values()) for d_2 in d.values()])

fig = go.Figure([go.Bar(x=list(d_sum.keys()), y=list(d_sum.values()))])

fig.update_layout(title_text="Comparision of structures' efficiency using summed time")
fig.update_yaxes(title="time [s]")
fig.show()

In [23]:
InputNames = [x.__name__ for x in inputTypes]
subplot_titles = []
for x in InputNames:
    for y in InputNames:
        subplot_titles.append(f"add:{x}, del:{y}".replace("generate",""))

fig = make_subplots(rows=8, cols=2,subplot_titles=subplot_titles)


i=0
for add_method in InputNames:
    for delete_method in InputNames:
        d = {}
        for struct, time in times.items():
            d[struct] = time[add_method][delete_method]
        fig.add_trace(go.Bar(x=list(d.keys()), y=list(d.values()), text=[round(x,3) for x in d.values()],
                            textposition="auto", name=""),
                      row=i//2+1, col=i%2+1)
        fig.update_yaxes(title_text="time [s]", row=i//2+1, col=i%2+1)
        i+=1

fig.update_layout(height=1600, width=1200,
                  title_text="Comparision of structures' efficiency based on addition and removal order")

fig.show()


### Conclusions:

1. Sorted (singly) linked list:
    - the easiest to implement
    - low memory usage
    - fast addition and deletion of nodes that are close to the head
2. Binary search tree:
    - more complex
    - often becomes degenerate that causes it to become even slower than our linked list
    - when it does not become degenerate it is faster than AVL since it doesnt use time for rotations
3. AVL tree (optionally):
    - the most complex (as it is an extension of the previous one)
    - overall the fastest structures and most consistent results
    - guaranteed O(logn) time complexity for every operation
    
